In [51]:
!pip install "protobuf==3.20.*"

In [ ]:
padded_path = "/kaggle/input/arabic-diacritics/keras/default/1/padded.pkl"
val_path = "/kaggle/input/val-2/padded_val.pkl"

diacritic2id_path = "/kaggle/input/arabic-diacritics/keras/default/1/diacritic2id.json"
idx2char_path = "/kaggle/input/arabic-diacritics/keras/default/1/idx2char.json"
char2idx_path = "/kaggle/input/arabic-diacritics/keras/default/1/char2idx.json"

In [3]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, TimeDistributed, Bidirectional
import json
import os
import pickle
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
import re

2025-11-26 11:30:10.791483: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764156611.024948      47 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764156611.087907      47 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [5]:
with open(diacritic2id_path, 'r', encoding='utf-8') as f:
    label2id = json.load(f)

id2label = {v: k for k, v in label2id.items()}

with open(char2idx_path, 'r', encoding='utf-8') as f:
    char2idx = json.load(f)

with open(idx2char_path, 'r', encoding='utf-8') as f:
    idx2char = json.load(f)

In [6]:
MAX_SEQ_LEN = 807
EMBEDDING_DIM = 32
LSTM_UNITS = 64
BATCH_SIZE = 32
EPOCHS = 10
VOCAB_SIZE = len(char2idx)
NUM_CLASSES = len(label2id)
DIRTY = False
DIACRITICS_PATTERN = re.compile(r'[\u064B-\u0652]')

In [49]:
def load_val_pickle(file_path):
    with open(file_path, 'rb') as f:
        X_raw, y_raw = pickle.load(f)
    X = []
    y = []

    X_txt = []
    y_label = []

    for text_seq, label_seq in zip(X_raw, y_raw):

        x_ids = [char2idx.get(c, char2idx['<PAD>']) for c in text_seq]
        y_ids = [label2id.get(t, 14) for t in label_seq]

        X.append(x_ids)
        y.append(y_ids)

    return np.array(X), np.array(y)



In [7]:
def load_data_pickle(file_path):
    with open(file_path, 'rb') as f:
        X_raw, y_raw = pickle.load(f)
    X = []
    y = []

    for text_seq, label_seq in zip(X_raw, y_raw):
        x_ids = [char2idx.get(c, char2idx['<PAD>']) for c in text_seq]
        y_ids = [label2id.get(t, 14) for t in label_seq]

        X.append(x_ids)
        y.append(y_ids)

    return np.array(X), np.array(y)



In [8]:
X, y = load_data_pickle(padded_path)

In [52]:
X_val, y_val = load_val_pickle(val_path)

In [9]:
if DIRTY:
    X = pad_sequences(X, maxlen=500, padding='post', truncating='post')
    y = pad_sequences(y, maxlen=500, padding='post', truncating='post')
    MAX_SEQ_LEN = 500

In [10]:
model = Sequential([
    Embedding(input_dim=len(char2idx), output_dim=EMBEDDING_DIM, input_length=MAX_SEQ_LEN), #,mask_zero=True),
    Bidirectional(LSTM(LSTM_UNITS, return_sequences=True)),
    TimeDistributed(Dense(len(label2id), activation='softmax'))
])

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
I0000 00:00:1764156655.172613      47 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1764156655.173402      47 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed                │ ?                      │   0 (unbuilt) │
│ (TimeDistributed)               │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [11]:
history = model.fit(
    X,
    y,
    batch_size=BATCH_SIZE,
    epochs=2,
    validation_split=0.1
)

Starting Training...
Epoch 1/2


I0000 00:00:1764156717.289805     114 cuda_dnn.cc:529] Loaded cuDNN version 90300


2258/2258 ━━━━━━━━━━━━━━━━━━━━ 597s 237ms/step - accuracy: 0.9462 - loss: 0.1786 - val_accuracy: 0.9785 - val_loss: 0.0645
Epoch 2/2
2258/2258 ━━━━━━━━━━━━━━━━━━━━ 522s 231ms/step - accuracy: 0.9805 - loss: 0.0585 - val_accuracy: 0.9840 - val_loss: 0.0482


In [12]:
model.save('tashkeel_bilstm.h5')
print("Model saved.")

Model saved.


In [13]:
def load_raw(file_path):
    with open(file_path, 'rb') as f:
        X_raw, y_raw = pickle.load(f)

    X_txt = []
    y_label = []

    for text_seq, label_seq in zip(X_raw, y_raw):
        X_txt.append(text_seq)
        y_label.append(label_seq)


    return X_txt, y_label

In [14]:
raw_texts, label_texts = load_raw(padded_path)

In [42]:
ground_truth = []
for test_idx in range(2000, 4000):
    result_str = ""
    for i in range(len(raw_texts[test_idx])):
        if raw_texts[test_idx][i] == '<PAD>':
            break
        result_str += raw_texts[test_idx][i] + label_texts[test_idx][i]
    ground_truth.append(result_str)

In [44]:
len(ground_truth)

4000

In [46]:
def split_text_and_diacritics(text):

    letters = []
    labels = []
    
    i = 0
    while i < len(text):
        char = text[i]
        
        if DIACRITICS_PATTERN.match(char):
            if labels:
                labels[-1] += char
        
        else:
            letters.append(char)
            labels.append("") 
            
        i += 1
        
    return "".join(letters), labels 

In [47]:
count = 0
tot_count = 0
for i in range(len(training_results)):
    _, labels_truth = split_text_and_diacritics(ground_truth[i])
    _, labels_predicted = split_text_and_diacritics(training_results[i])
    for j in range(len(labels_truth)):
        if labels_truth[j] == labels_predicted[j]:
            count += 1
        tot_count += 1
print(count / tot_count)

0.9060922776205619


In [41]:
training_results = []
for test_idx in range(2000, 4000):
    sample_input = X[test_idx]
    prediction = model.predict(np.array([sample_input]))
    pred_ids = np.argmax(prediction, axis=-1)[0]
    
    input_chars = raw_texts[test_idx] 
    
    result_str = ""
    for char, p_id in zip(input_chars, pred_ids):
        if char == '<PAD>':
            break
    
        diacritic = id2label.get(p_id, "")
        result_str += char + diacritic
    
    training_results.append(result_str)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
1/1 ━━━━━━━━

In [53]:
raw_val, label_val = load_raw(val_path)

In [61]:
def predict_val(model, X, raw_texts):
    val_results = []
    for i, x in enumerate(X[:2000]):
        sample_input = x
        prediction = model.predict(np.array([sample_input]))
        pred_ids = np.argmax(prediction, axis=-1)[0]
        
        input_chars = raw_texts[i] 
        
        result_str = ""
        for char, p_id in zip(input_chars, pred_ids):
            if char == '<PAD>':
                break
        
            diacritic = id2label.get(p_id, "")
            result_str += char + diacritic
        
        val_results.append(result_str)
    return val_results

In [62]:
val_results = predict_val(model, X_val, raw_val)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
1/1 ━━━━━━━━

In [64]:
def get_val_ground_truth(raw_texts, label_texts):
    ground_val_truth = []
    for ind, raw_text in enumerate(raw_texts[:2000]):
        result_str = ""
        for i in range(len(raw_text)):
            if raw_text[i] == '<PAD>':
                break
            result_str += raw_text[i] + label_texts[ind][i]
        ground_val_truth.append(result_str)

    return ground_val_truth

In [65]:
ground_val_truth = get_val_ground_truth(raw_val, label_val)

In [66]:

count_val = 0
tot_count_val = 0
for i in range(len(val_results)):
    _, labels_truth = split_text_and_diacritics(ground_val_truth[i])
    _, labels_predicted = split_text_and_diacritics(val_results[i])
    for j in range(len(labels_truth)):
        if labels_truth[j] == labels_predicted[j]:
            count_val += 1
        tot_count_val += 1
print(count_val / tot_count_val)

0.905658865312813
